<a href="https://colab.research.google.com/github/douglasbarbosadelima/Data-Science/blob/master/w2v_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Utilização de uma camada embedding com keras..ainda yelp..polaridade

In [1]:
!pip install tensorflow==2.0.0 

     |████████████████████████████████| 86.3MB 69kB/s 
     |████████████████████████████████| 450kB 48.9MB/s 
     |████████████████████████████████| 3.8MB 28.3MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
import numpy as np
import random

In [4]:
df=pd.read_csv('yelp_short_10k.csv')
df.head()

,Unnamed: 0,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


In [5]:
df['stars'].value_counts()

5    3554
4    2966
3    1716
2     891
1     873
Name: stars, dtype: int64

Para fazermos uma classificação binária entre ótimo ou não, vamos considerar ótimo apenas  4 e o 5 e não ótimo os outros. Isso vai "balancear as classes"

In [0]:
df['classe']=0
filtro=df[df['stars']>=4].index
df.loc[filtro,'classe']=1

In [7]:
df['classe'].value_counts()

1    6520
0    3480
Name: classe, dtype: int64

Vamos tirar "aleatoriamente" 6520-3480 amostras de classe 1 (para simplificar, poderíamos usar critérios mais sofisticados para reduzir a amostra positiva)

In [0]:
index_1=df[df['classe']==1].index
index_1=list(index_1)
index_1
index_1_2=random.sample(index_1,k=3040)
df.drop(index=index_1_2,axis=0,inplace=True)

In [9]:
df['classe'].value_counts()

1    3480
0    3480
Name: classe, dtype: int64

In [52]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
vect=CountVectorizer(stop_words=stop,binary=True,max_features=8000,ngram_range=(1,2))
vect.fit(df['text'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=8000, min_df=1,
                ngram_range=(1, 2), preprocessor=None,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...},
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [53]:
print(len(vect.vocabulary_),vect.vocabulary_)

8000 {'super': 6820, 'simple': 6362, 'place': 5190, 'amazing': 288, 'nonetheless': 4698, 'around': 411, 'since': 6365, '30': 58, 'still': 6710, 'serve': 6175, 'thing': 7062, 'started': 6667, 'salami': 5990, 'sandwich': 6017, 'mustard': 4545, 'staff': 6626, 'helpful': 3371, 'friendly': 2789, 'place amazing': 5193, 'staff helpful': 6631, 'helpful friendly': 3372, 'small': 6434, 'changes': 1182, 'menu': 4355, 'every': 2291, 'often': 4771, 'cool': 1543, 'decor': 1804, 'vibe': 7538, 'inside': 3612, 'seat': 6111, 'restaurant': 5831, 'call': 1029, 'reservation': 5817, 'beef': 679, 'tartar': 6960, 'pork': 5344, 'belly': 704, 'start': 6664, 'salmon': 5994, 'dish': 1965, 'lamb': 3800, 'meal': 4302, 'mains': 4187, 'everything': 2307, 'incredible': 3576, 'could': 1572, 'go': 2981, 'length': 3890, 'listed': 3989, 'ingredients': 3600, 'really': 5658, 'make': 4197, 'dishes': 1969, 'honestly': 3442, 'need': 4579, 'bit': 777, 'outside': 4921, 'downtown': 2015, 'montreal': 4489, 'take': 6926, 'metro': 4

In [55]:
#geração dos inputs 
#vou pegando o número do vocab para cada palavra e fazendo listas 
tam=20
y2=[]
inputx=[]
quant=[]
for i in range(len(df)):
  frase=[]
  v=vect.transform([df.iloc[i,6]])
  va=v.toarray()
  vb=va[0]
  quant=vb.sum()
  if quant>=tam:
    y2.append(df.iloc[i,10])
    k=0
    while k<len(vb):
      if vb[k]>0:frase.append(k)
      k+=1
    inputx.append(np.array(frase[:tam]))
print(inputx[0])

[  58  288  411 2789 3371 3372 4545 4698 5190 5193 5990 6017 6175 6362
 6365 6626 6631 6667 6710 6820]


In [80]:
# Find index word and generate output.

keys = list(vect.vocabulary_.keys())
values = list(vect.vocabulary_.values())

[keys[values.index(i)] for i in inputx[10]]

['absolutely',
 'almost',
 'anymore',
 'awful',
 'back',
 'back make',
 'beer',
 'belly',
 'better',
 'better experience',
 'bites',
 'bland',
 'bun',
 'came',
 'came two',
 'care',
 'cold',
 'coming',
 'could',
 'eating']

In [60]:
tam=[len(x) for x in inputx]
max(tam),min(tam),sum(tam)/len(tam)

(20, 20, 20.0)

In [0]:
inputx=np.array(inputx)
y2=np.array(y2)

In [47]:
inputx[0].shape

(20,)

In [0]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(8000, 200, input_length=20))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [63]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 200)           1600000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 4000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 4001      
Total params: 1,604,001
Trainable params: 1,604,001
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [65]:
model.fit(inputx,y2,epochs=80,validation_split=0.3)

Train on 4243 samples, validate on 1819 samples
Epoch 1/80
4243/4243 [==============================] - 3s 791us/sample - loss: 0.6668 - accuracy: 0.6149 - val_loss: 0.6059 - val_accuracy: 0.7394
Epoch 2/80
4243/4243 [==============================] - 3s 624us/sample - loss: 0.4125 - accuracy: 0.8763 - val_loss: 0.5178 - val_accuracy: 0.7482
Epoch 3/80
4243/4243 [==============================] - 3s 627us/sample - loss: 0.1651 - accuracy: 0.9774 - val_loss: 0.5414 - val_accuracy: 0.7339
Epoch 4/80
4243/4243 [==============================] - 3s 631us/sample - loss: 0.0559 - accuracy: 0.9993 - val_loss: 0.5734 - val_accuracy: 0.7306
Epoch 5/80
4243/4243 [==============================] - 3s 625us/sample - loss: 0.0244 - accuracy: 1.0000 - val_loss: 0.6040 - val_accuracy: 0.7284
Epoch 6/80
4243/4243 [==============================] - 3s 624us/sample - loss: 0.0137 - accuracy: 1.0000 - val_loss: 0.6306 - val_accuracy: 0.7268
Epoch 7/80
4243/4243 [==============================] - 3s 625us

In [74]:
#fazer a predição para "10* nice + 10*loved"
#tentar melhorar o modelo

model.predict([[5654]*20])

array([[0.55700016]], dtype=float32)